In [1]:
import random
import numpy as np
import tensorflow as tf

from src.features.encodings import pse_knc

random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [2]:
from pathlib import Path
from keras import Sequential
from keras.layers import *
from xgboost import XGBClassifier
from keras.callbacks import EarlyStopping
from keras.losses import BinaryCrossentropy

from src.features.encoder import FeatureEncoder
from src.data.data_loader import load_dataset, Species
from src.models.reporting.model_report import ModelReport
from src.models.reporting.single_report import generate_report

### Human Training & Testing

In [7]:
human_train = load_dataset(
    Species.human,
    encoding=lambda x: FeatureEncoder.pstnpss(x, 'hs')
)

human_test = load_dataset(
    Species.human,
    independent=True,
    encoding=lambda x: FeatureEncoder.pstnpss(x, 'hs')
)

In [24]:
human_test[0].shape

(200, 19)

In [61]:
def create_weak_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=19))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create a list to store weak models
weak_models = []

# Create and train individual weak models
num_weak_models = 5  # Number of weak models to train
for _ in range(num_weak_models):
    model = create_weak_model()
    # Train the weak model
    model.fit(human_train[0], human_train[1], epochs=10, batch_size=32, verbose=0, callbacks=[EarlyStopping(patience=3)])
    weak_models.append(model)

# Make predictions using the ensemble
ensemble_predictions = []
for model in weak_models:
    predictions = model.predict(human_test[0])
    ensemble_predictions.append(predictions)

# Combine predictions using majority voting
ensemble_predictions = np.concatenate(ensemble_predictions, axis=1)
ensemble_predictions = (ensemble_predictions > 0.5).astype(int)
final_predictions = np.mean(ensemble_predictions, axis=1)

# Calculate accuracy
accuracy = np.mean(final_predictions == human_test[1])
print("Ensemble Model Accuracy:", accuracy)

7/7 [==============================] - 0s 504us/step
Ensemble Model Accuracy: 0.755


In [42]:
human_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(19, 1)),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

human_model.compile(
    loss=BinaryCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy']
)

human_model.fit(human_train[0], human_train[1], epochs=100, batch_size=32,
                validation_data=(human_test[0], human_test[1]))

Epoch 1/100
31/31 [==============================] - 0s 4ms/step - loss: 0.6897 - accuracy: 0.5495 - val_loss: 0.6779 - val_accuracy: 0.7600
Epoch 2/100
31/31 [==============================] - 0s 2ms/step - loss: 0.6710 - accuracy: 0.6364 - val_loss: 0.6313 - val_accuracy: 0.7000
Epoch 3/100
31/31 [==============================] - 0s 2ms/step - loss: 0.6358 - accuracy: 0.6677 - val_loss: 0.5705 - val_accuracy: 0.7300
Epoch 4/100
31/31 [==============================] - 0s 2ms/step - loss: 0.6146 - accuracy: 0.6606 - val_loss: 0.5457 - val_accuracy: 0.7600
Epoch 5/100
31/31 [==============================] - 0s 2ms/step - loss: 0.6128 - accuracy: 0.6576 - val_loss: 0.5672 - val_accuracy: 0.6950
Epoch 6/100
31/31 [==============================] - 0s 2ms/step - loss: 0.6051 - accuracy: 0.6707 - val_loss: 0.5278 - val_accuracy: 0.7450
Epoch 7/100
31/31 [==============================] - 0s 2ms/step - loss: 0.6027 - accuracy: 0.6657 - val_loss: 0.5435 - val_accuracy: 0.7150
Epoch 8/100
3

In [60]:

human_model.fit(human_train[0], human_train[1], epochs=40, batch_size=32,
                validation_data=(human_test[0], human_test[1]))

Epoch 1/40
31/31 [==============================] - 0s 3ms/step - loss: 0.1497 - accuracy: 0.9394 - val_loss: 1.1844 - val_accuracy: 0.7200
Epoch 2/40
31/31 [==============================] - 0s 2ms/step - loss: 0.1230 - accuracy: 0.9586 - val_loss: 1.2192 - val_accuracy: 0.7350
Epoch 3/40
31/31 [==============================] - 0s 2ms/step - loss: 0.1202 - accuracy: 0.9556 - val_loss: 1.2192 - val_accuracy: 0.7250
Epoch 4/40
31/31 [==============================] - 0s 2ms/step - loss: 0.1343 - accuracy: 0.9535 - val_loss: 1.1852 - val_accuracy: 0.7250
Epoch 5/40
31/31 [==============================] - 0s 2ms/step - loss: 0.1284 - accuracy: 0.9515 - val_loss: 1.2307 - val_accuracy: 0.7300
Epoch 6/40
31/31 [==============================] - 0s 2ms/step - loss: 0.1472 - accuracy: 0.9434 - val_loss: 1.1983 - val_accuracy: 0.7150
Epoch 7/40
31/31 [==============================] - 0s 2ms/step - loss: 0.1218 - accuracy: 0.9535 - val_loss: 1.2809 - val_accuracy: 0.7250
Epoch 8/40
31/31 [==

### Yeast Training & Testing

In [11]:
yeast_x_test, yeast_y_test = load_dataset(Species.yeast, independent=True,
                                          encoding=lambda x: FeatureEncoder.pstnpss(x, 'sc'))
yeast_x_train, yeast_y_train = load_dataset(Species.yeast, independent=False,
                                            encoding=lambda x: FeatureEncoder.pstnpss(x, 'sc'))

In [14]:
yeast_model = Sequential([
    Dense(29, input_dim=29, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

yeast_model.compile(loss=BinaryCrossentropy(from_logits=False), optimizer='adam', metrics=['accuracy'])

In [16]:
yeast_model.fit(yeast_x_train, yeast_y_train, epochs=40, callbacks=[early_stopping],
                validation_data=(yeast_x_test, yeast_y_test))

Epoch 1/40
20/20 [==============================] - 0s 2ms/step - loss: 0.3498 - accuracy: 0.8678 - val_loss: 0.3375 - val_accuracy: 0.8400
Epoch 2/40
20/20 [==============================] - 0s 1ms/step - loss: 0.3496 - accuracy: 0.8631 - val_loss: 0.3232 - val_accuracy: 0.8400
Epoch 3/40
20/20 [==============================] - 0s 1ms/step - loss: 0.3691 - accuracy: 0.8567 - val_loss: 0.3369 - val_accuracy: 0.8400
Epoch 4/40
20/20 [==============================] - 0s 1ms/step - loss: 0.3359 - accuracy: 0.8646 - val_loss: 0.3245 - val_accuracy: 0.8400
Epoch 5/40
20/20 [==============================] - 0s 1ms/step - loss: 0.3456 - accuracy: 0.8615 - val_loss: 0.3265 - val_accuracy: 0.8400
Epoch 6/40
20/20 [==============================] - 0s 1ms/step - loss: 0.3616 - accuracy: 0.8567 - val_loss: 0.3297 - val_accuracy: 0.8350
Epoch 7/40
20/20 [==============================] - 0s 1ms/step - loss: 0.3509 - accuracy: 0.8631 - val_loss: 0.3217 - val_accuracy: 0.8450
Epoch 8/40
20/20 [==

In [17]:
human_model_report = ModelReport.generate(human_model, human_x_test, human_y_test, is_keras=True)
yeast_model_report = ModelReport.generate(yeast_model, yeast_x_test, yeast_y_test, is_keras=True)

7/7 [==============================] - 0s 507us/step


In [18]:
generate_report({Species.human.value: human_model_report, Species.yeast.value: yeast_model_report},
                Path('nn/pstnpss_1'))

note: Running TeX ...
note: Rerunning TeX because "report.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `nn/pstnpss_1/report.pdf` (33.19 KiB)
note: Skipped writing 1 intermediate files (use --keep-intermediates to keep them)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [20]:
def select_features(features):
    selection = [10, 12, 8, 9, 17, 7, 6, 15, 14, 11, 2, 13, 5, 16, 18, 4, 1, 3, 0]
    imp_features = list(map(lambda x: f'pstnpss_{x + 1}', selection))
    return features[imp_features]

In [21]:
human_classifier = XGBClassifier(
    base_score=0.5, booster='gbtree', colsample_bynode=1, max_depth=6, verbosity=1, colsample_bytree=0.637482,
    subsample=0.901284, learning_rate=0.276002, reg_alpha=0, max_delta_step=0, min_child_weight=1, n_jobs=1,
    n_estimators=1082, colsample_bylevel=1, random_state=0, reg_lambda=1, scale_pos_weight=1, gamma=0.103823
)

human_classifier.fit(select_features(human_x_train), human_y_train)
human_classifier.score(select_features(human_x_test), human_y_test)

0.8

In [22]:
yeast_classifier = XGBClassifier(
    base_score=0.5, booster='gbtree', colsample_bynode=1, max_depth=6, verbosity=1, colsample_bytree=0.637482,
    subsample=0.901284, learning_rate=0.276002, reg_alpha=0, max_delta_step=0, min_child_weight=1, n_jobs=1,
    n_estimators=1082, colsample_bylevel=1, random_state=0, reg_lambda=1, scale_pos_weight=1, gamma=0.103823
)

yeast_classifier.fit(select_features(yeast_x_train), yeast_y_train)
yeast_classifier.score(select_features(yeast_x_test), yeast_y_test)

0.775

In [23]:
human_model_report = ModelReport.generate(human_classifier, select_features(human_x_test), human_y_test)
yeast_model_report = ModelReport.generate(yeast_classifier, select_features(yeast_x_test), yeast_y_test)

In [24]:
generate_report({Species.human.value: human_model_report, Species.yeast.value: yeast_model_report},
                Path('ml/pstnpss_1'))

note: Running TeX ...
note: Rerunning TeX because "report.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `ml/pstnpss_1/report.pdf` (32.92 KiB)
note: Skipped writing 1 intermediate files (use --keep-intermediates to keep them)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>